# Speeding up your code using `functools.lru_cache`

## What is memoization?
Memoization simply means adding some memory to your function, so that it doesn't need to compute the same result twice. It's very useful when the function gets called a lot with the same inputs and when such calls are expensive to compute.

## What is `functools.lru_cache`?
It's a decorator from the `functools` module that lets you use memoization easily. It creates a dictionary that associates inputs and outputs of your function. That dictionary acts as a cache memory. We will see how to use this decorator below.

## The Tower of Hanoi
As an example, we will use a function that computes a solution to the Tower of Hanoi puzzle, which is stated as follows (see [Wikipedia – Tower of Hanoi](https://en.wikipedia.org/wiki/Tower_of_Hanoi#Simpler_statement_of_iterative_solution)):

The Tower of Hanoi is a mathematical puzzle. It consists of three rods and a number of disks of different sizes, which can slide onto any rod. The puzzle starts with the disks in a neat stack in ascending order of size on one rod, the smallest at the top, thus making a conical shape.

The objective of the puzzle is to move the entire stack to another rod, obeying the following simple rules:
1. Only one disk can be moved at a time.
2. Each move consists of taking the upper disk from one of the stacks and placing it on top of another stack or on an empty rod.
3. No larger disk may be placed on top of a smaller disk.

![Image of the Tower of Hanoi problem](https://upload.wikimedia.org/wikipedia/commons/0/07/Tower_of_Hanoi.jpeg)

## Recursive solution to the Tower of Hanoi
We will use a simple recursive solution to this puzzle. Let's name the rods "A", "B" and "C" from left to right. Starting with $n$ disks on rod "A", we first move the $n-1$ smallest disks to rod "B". How do we do that? We don't have to know it: that's the magic of recursive solutions! Then, we move the largest disk from rod "A" to rod "C". Finally, we move the $n-1$ smallest disks from rod "B" to rod "C". This solution is further detailed on [Wikipedia](https://en.wikipedia.org/wiki/Tower_of_Hanoi#Recursive_solution). Let's implement it!

In [5]:
from functools import lru_cache  # We will need this later on
from pprint import pprint        # Useful to print things nicely
from time import time            # Useful to time our function


def hanoi(n_disks, start_pole, free_pole, end_pole):
    # What are the steps to move 'n_disks' disks from 'start_pole' to 'end_pole'?
    steps = list()
    
    if n_disks > 0:
        # Move all disks except the biggest one from 'start_pole' to 'free_pole'
        steps.extend(hanoi(n_disks - 1, start_pole, end_pole, free_pole))

        # Move biggest disk from 'start_pole' to 'end_pole'
        steps.append(f"Move disk {n_disks} from rod {start_pole} to rod {end_pole}")

        # Move all disks except the biggest one from 'free_pole' to 'end_pole'
        steps.extend(hanoi(n_disks - 1, free_pole, start_pole, end_pole))

    return steps

Let's test this function with a small amount of disks. Disk 1 is the smallest one and disk 3 is the largest one. Try to follow the steps to check that its correct!

In [6]:
pprint(hanoi(3, "A", "B", "C"))

['Move disk 1 from rod A to rod C',
 'Move disk 2 from rod A to rod B',
 'Move disk 1 from rod C to rod B',
 'Move disk 3 from rod A to rod C',
 'Move disk 1 from rod B to rod A',
 'Move disk 2 from rod B to rod C',
 'Move disk 1 from rod A to rod C']


To decrease the length of the output, we will limit ourselves to simply count the number of steps required. There exists a simple formula to do that, but we won't use it so that we can study memoization with a simple example. In case you're wondering, the formula is $steps(n) = 2^n - 1$. Let's rewrite our function to only count the number of steps.

We will also time it to keep track of how it performs. We will be using the `time` module instead of the `timeit` module, because we don't need much precision for today's purpose.

In [3]:
def hanoi_count(n_disks, start_pole, free_pole, end_pole):
    # How many steps to move 'n_disks' disks from 'start_pole' to 'end_pole'?
    
    steps = 0    
    if n_disks > 0:
        # Move all disks except the biggest one from 'start_pole' to 'free_pole'
        steps += hanoi_count(n_disks - 1, start_pole, end_pole, free_pole)

        # Move biggest disk from 'start_pole' to 'end_pole'
        steps += 1

        # Move all disks except the biggest one from 'free_pole' to 'end_pole'
        steps += hanoi_count(n_disks - 1, free_pole, start_pole, end_pole)

    return steps


n_disks = 24

start = time()
n_steps = hanoi_count(n_disks, "A", "B", "C")
stop = time()

print(f"The number of steps for {n_disks} disks is {n_steps} (exec.: {round(stop - start, 3)} seconds)")

The number of steps for 24 disks is 16777215 (exec.: 6.279 seconds)


## The issue with the recursive solution
This solution is nice because it is extremely simple. However, it creates a lot of recursive calls when the number of disks increases. You can check it by calling the function `hanoi_count` with 30 disks for example. There are also many calls that have the exact same arguments. That's exactly the problem that memoization helps to solve.

## Adding memoization
To add memoization, you just need to add `@lru_cache()` above the definition of your function.

In [7]:
@lru_cache()
def hanoi_count(n_disks, start_pole, free_pole, end_pole):
    # How many steps to move 'n_disks' disks from 'start_pole' to 'end_pole'?
    
    steps = 0    
    if n_disks > 0:
        # Move all disks except the biggest one from 'start_pole' to 'free_pole'
        steps += hanoi_count(n_disks - 1, start_pole, end_pole, free_pole)

        # Move biggest disk from 'start_pole' to 'end_pole'
        steps += 1

        # Move all disks except the biggest one from 'free_pole' to 'end_pole'
        steps += hanoi_count(n_disks - 1, free_pole, start_pole, end_pole)

    return steps


n_disks = 350

start = time()
n_steps = hanoi_count(n_disks, "A", "B", "C")
stop = time()

print(f"The number of steps for {n_disks} disks is {n_steps} (exec.: {round(stop - start, 3)} seconds)")

The number of steps for 350 disks is 2293498615990071511610820895302086940796564989168281123737588839386922876088484808070018553110125686554623 (exec.: 0.001 seconds)


As you can see, it took about $5$ seconds to get the solution for $24$ disks using the regular `hanoi_count` function and it took a $1$ millisecond to get the solution for $350$ disks when we added memoization. This is a **huge** difference. The execution time of the regular function actually doubles when the number of disks increases by one. This means that for $350$ disks, it would take about $6.84\cdot 10^{98}$ seconds, which is more than billions and billions and billions of times the age of the entire universe.

To sum it up: memoization is nice.

Further details are given in the [documentation](https://docs.python.org/3.8/library/functools.html) of the `functools` module. For example, you can change the amount of cached calls you allow using the `maxsize` parameter like so: `@lru_cache(maxsize=10)`.

Thanks for reading, I hope you learned something!